In [11]:
import torch
import torch.nn.functional as F
from torch.utils import data
import matplotlib.pyplot as plt  # for making figures
%matplotlib inline

In [12]:
words = open('indian-names.txt', 'r').read().splitlines()

In [13]:
words[:10]

['aabid',
 'aabida',
 'aachal',
 'aadesh',
 'aadil',
 'aadish',
 'aaditya',
 'aaenab',
 'aafreen',
 'aafrin']

In [14]:
# build the vocabulary of characters and mappings to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

In [15]:
# building the dataset

def build_dataset(words):
    block_size = 3  # context length: how many characters do we take to predict the next one?
    X, Y = [], []  # X is the array of the input characters, Y is the output for those input characters
    
    for w in words:
        
        # print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '---->', ch)
            context = context[1:] + [ix]  # crop and append for next character's context
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([38071, 3]) torch.Size([38071])
torch.Size([4741, 3]) torch.Size([4741])
torch.Size([4749, 3]) torch.Size([4749])


In [16]:
# dataset
Xtr.shape, Ytr.shape

(torch.Size([38071, 3]), torch.Size([38071]))

In [17]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)   
b1 = torch.randn((200), generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn((27), generator=g)
parameters = [C, W1, b1, W2, b2]

In [19]:
for p in parameters:
    p.requires_grad = True

sum(p.nelement() for p in parameters)  # number of parameters in total

11897

In [20]:
lri = []
lossi = []
stepi = []

In [21]:
for i in range(200000):

    # minibatch construction
    ix = torch.randint(0, Xtr.shape[0], (64,))  # generates 32 numbers lying between 0 and X.shape[0]
    
    # forward pass
    emb = C[Xtr[ix]]  # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)  # (32, 100)
    logits = h @ W2 + b2  # (32, 27)
    loss = F.cross_entropy(logits, Ytr[ix])  # does the older three steps in one and is more efficient 
    # print(loss.item())  # but this is only the loss for that many batch
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    # lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.log10().item())

In [22]:
print(loss.item())

1.9695833921432495


In [23]:
# train set loss
# for evaluating the full loss

emb = C[Xtr]  # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2  # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

tensor(1.8698, grad_fn=<NllLossBackward0>)

In [24]:
# dev set

emb = C[Xdev]  # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)  # (32, 100)
logits = h @ W2 + b2  # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1458, grad_fn=<NllLossBackward0>)

In [28]:
# sample from the model

g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    block_size = 3
    out = []
    context = [0] * block_size  # initialize with all '...'
    while True:
        emb = C[torch.tensor([context])]  # (1, block_size, d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

mohabim.
anushel.
ndhramparvturul.
jaram.
radeep.
seli.
hemparen.
misingh.
parshamtekchana.
misshubhrgahima.
saki.
jowadhila.
bhudayubeen.
deviyajul.
mho.
kar.
mustkila.
har.
sakyarsu.
punjal.
